In [ ]:
import urllib
import urllib.parse
#get - method1
# response = urllib.request.urlopen('https://www.baidu.com')
# print(response.read().decode('utf-8')) #对获取的网页进行utf-8解码
#post - method2 httpbin.org
data = bytes(urllib.parse.urlencode({"hello":"world"}),encoding='utf-8')
postresponse = urllib.request.urlopen('http://www.httpbin.org/post',data=data)
print(postresponse.read().decode('utf-8'))

In [2]:
import urllib
# try:
#     getresponse = urllib.request.urlopen('http://www.httpbin.org/get',timeout=0.01)
#     print(getresponse.read().decode('utf-8'))
# except urllib.error.URLError as e:
#     print("time out!")

In [3]:
getresponse = urllib.request.urlopen('http://www.httpbin.org/get')
print(getresponse.status)

200


In [4]:
# response = urllib.request.urlopen("http://douban.com")
# print(reponse.status)
#error 418 证明对方知道你是一个查重

In [5]:
response = urllib.request.urlopen("http://www.baidu.com")
print(response.getheader("Server"))

BWS/1.1


In [8]:
url2 = "http://httpbin.org/post"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75"
}
data = bytes(urllib.parse.urlencode({"name":"Eric"}),encoding="utf-8")
req = urllib.request.Request(url=url,data=data,headers=headers,method ="POST")
res = urllib.request.urlopen(req)
print(res.read().decode("utf-8"))

<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8" />
<title></title>
<link rel="stylesheet" href="//dragonstatic.com/parking/css/native.css" type="text/css" media="screen" />
<script type="059d933767d5ef4a81979b6e-text/javascript" src="//dragonstatic.com/parking/js/track.js?v=190929"></script>
<script type="059d933767d5ef4a81979b6e-text/javascript">var pageinfo = {ad:1};</script>
<script type="059d933767d5ef4a81979b6e-text/javascript" src="//dragonstatic.com/parking/js/native.js?v=201206"></script>

</head>
<body>
<div id='root'></div>
<script src="https://ajax.cloudflare.com/cdn-cgi/scripts/7089c43e/cloudflare-static/rocket-loader.min.js" data-cf-settings="059d933767d5ef4a81979b6e-|49" defer=""></script></body>
</html>



In [9]:
url = "https://www.doubai.com"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75"
}
req = urllib.request.Request(url=url,headers=headers)
res = urllib.request.urlopen(req)
print(res.read().decode("utf-8"))

<!DOCTYPE html><html data-adblockkey="MFwwDQYJKoZIhvcNAQEBBQADSwAwSAJBANDrp2lz7AOmADaN8tA50LsWcjLFyQFcb/P2Txc58oYOeILb3vBw7J6f4pamkAQVSQuqYsKx3YzdUHCvbVZvFUsCAwEAAQ==_m03SKooon2mopHAMPek9H6B2+rng2YoCq9vX8nWjgoLHN7+dVEM56JL6xHmifeporw4PG49tmDn6JM1QYwbU0Q=="><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><title></title><meta name="viewport" content="width=device-width, initial-scale=1"><meta name="description" content="See related links to what you are looking for."/></head><!--[if IE 6 ]><body class="ie6"><![endif]--><!--[if IE 7 ]><body class="ie7"><![endif]--><!--[if IE 8 ]><body class="ie8"><![endif]--><!--[if IE 9 ]><body class="ie9"><![endif]--><!--[if (gt IE 9)|!(IE)]> --><body><!--<![endif]--><script type="text/javascript">g_pb=(function(){var
DT=document,azx=location,DD=DT.createElement('script'),aAC=false,LU;DD.defer=true;DD.async=true;DD.src="//www.google.com/adsense/domains/caf.js";DD.onerror=function(){if(azx.search!=='?z'){azx.href='/?z';}};DD.onlo

In [2]:
import urllib
import re
from bs4 import BeautifulSoup as BS
import xlwt
import sqlite3
import os

def main():
    baseurl = "https://movie.douban.com/top250?start="
    #爬取网页
    datalist = getData(baseurl)
    savepath = "doubanTop250.xls"
    dbpath = "movie.db"
    #保存数据
    saveData(datalist,savepath)
    saveDataDb(datalist,dbpath)

#影片详情链接的规则
findlink = re.compile(r'<a href="(.*?)">') #创建一个正则表达式对象，表示规则（字符串的模式）
#影片图片链接的规则
findImg = re.compile(r'<img.*src="(.*?)"',re.S)#忽略换行符，匹配到其他的字符，所有出现的情况
#影片的片名
findTitle = re.compile(r'<span class="title">(.*)</span>')
#影片评分
findRating = re.compile(r'<span class="rating_num" property="v:average">(.*)</span>')
#评价人数
findJudge = re.compile(r'<span>(\d*)评价人</span>')
#找到概况
findInq = re.compile(r'<span class="inq">(.*)</span>')
#找到影片的相应内容
findBd = re.compile(r'<p class="">(.*?)</p>',re.S)

#爬取网页
def getData(baseurl):
    datalist = []
    for i in range(0,10): #调用获取页面信息的函数10次
        url = baseurl+str(i*25)
        html = askUrl(url) #保存获取到的网页源码
        #逐一解析数据
        souphtml = BS(html,"html.parser")
        for item in souphtml.find_all('div',class_="item"): #查找符合要求的字符串形成一个列表
            #class_表示的是属性值
            #print(item) #测试电影item的全部信息
            data = [] #保存一部电影的所有信息
            item = str(item)
            link = re.findall(findlink,item)[0] #re库通过正则表达式查找指定的字符串
            data.append(link)
            imgSrc = re.findall(findImg,item)[0]
            data.append(imgSrc)
            titles = re.findall(findTitle,item) #片名可能只有个中文名和外文名，或者只有一个
            if titles == 2:
                ctitle = titles[0]
                data.append(ctitle)
                otitle = titles[1].replace("/","")
                data.append(otitle)
            else:
                data.append(titles[0])
                data.append(' ')
            Rating = re.findall(findRating,item)[0]
            data.append(Rating)
            Judge = re.findall(findJudge,item)
            if len(Judge) == 1:
                data.append(Judge)
            else:
                data.append(" ")   
            Inq = re.findall(findInq,item)
            if len(Inq) != 0:
                Inq = Inq[0].replace("。","")
                data.append(Inq)
            else:
                data.append(" ")
            Bd = re.findall(findBd,item)[0]
            Bd = re.sub('<br(\s+)?/>(\s+)?'," ",Bd)
            Bd = re.sub('/'," ",Bd)
            data.append(Bd.strip())#去掉前后的空格
            datalist.append(data) #把处理好的一部电影数据放入datalist

    return datalist

#得到指定网页的一个内容
def askUrl(url):
    #让对方服务器认为自己是个浏览器
    header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75"
    }
    request = urllib.request.Request(url,headers=header);
    html = ""
    try:
        response = urllib.request.urlopen(request)
        html = response.read().decode("utf-8")
    except urllib.error.URLError as e:
        if hasattr(e,"code"):
            print(e.code)
        if hasattr(e,"reason"):
            print(e.reason)
    return html

#保存数据
def saveData(datalist,savePath):
    print("begin to save......")
    book = xlwt.Workbook(encoding = "utf-8",style_compression=0)
    sheet = book.add_sheet("doubanTop250",cell_overwrite_ok=True)
    col = ("电影详情链接","图片链接","影片中文名","影片外文名","评分","评价数","概况","相关信息")
    for i in range(0,8):
        sheet.write(0,i,col[i])
    for i in range(0,250):
        print("第%d条"%i)
        data = datalist[i]
        for j in range(0,8):
            sheet.write(i+1,j,data[j])
    book.save(savePath)
    print("save success!")

def init_db(dbpath):
    conn = sqlite3.connect(dbpath)
    cursor = conn.cursor()
    #创建数据表
    sql ='''
        create table movie250
        (
            id integer primary key autoincrement,
            info_link text,
            pic_link text,
            cname text,
            ename text,
            score text,
            rated text,
            introduction text,
            info text
        )
    '''
    cursor.execute(sql)
    conn.commit()
    conn.close()

def saveDataDb(datalist,dbpath):
    if os.path.exists("movie.db"):
        pass
    else:
        init_db(dbpath)
    conn = sqlite3.connect(dbpath)
    cur = conn.cursor()
    for data in datalist:
        for index in range(len(data)):
            data[index] = '"'+data[index]+'"'
        sql = '''
                insert into movie250(info_link,pic_link,cname,ename,score,rated,introduction,info)
                values(%s)'''%((",").join(data))
        print(sql)
        cur.execute(sql)
        conn.commit()
    conn.close()
if __name__ == "__main__": 
    main()

begin to save......
第0条
第1条
第2条
第3条
第4条
第5条
第6条
第7条
第8条
第9条
第10条
第11条
第12条
第13条
第14条
第15条
第16条
第17条
第18条
第19条
第20条
第21条
第22条
第23条
第24条
第25条
第26条
第27条
第28条
第29条
第30条
第31条
第32条
第33条
第34条
第35条
第36条
第37条
第38条
第39条
第40条
第41条
第42条
第43条
第44条
第45条
第46条
第47条
第48条
第49条
第50条
第51条
第52条
第53条
第54条
第55条
第56条
第57条
第58条
第59条
第60条
第61条
第62条
第63条
第64条
第65条
第66条
第67条
第68条
第69条
第70条
第71条
第72条
第73条
第74条
第75条
第76条
第77条
第78条
第79条
第80条
第81条
第82条
第83条
第84条
第85条
第86条
第87条
第88条
第89条
第90条
第91条
第92条
第93条
第94条
第95条
第96条
第97条
第98条
第99条
第100条
第101条
第102条
第103条
第104条
第105条
第106条
第107条
第108条
第109条
第110条
第111条
第112条
第113条
第114条
第115条
第116条
第117条
第118条
第119条
第120条
第121条
第122条
第123条
第124条
第125条
第126条
第127条
第128条
第129条
第130条
第131条
第132条
第133条
第134条
第135条
第136条
第137条
第138条
第139条
第140条
第141条
第142条
第143条
第144条
第145条
第146条
第147条
第148条
第149条
第150条
第151条
第152条
第153条
第154条
第155条
第156条
第157条
第158条
第159条
第160条
第161条
第162条
第163条
第164条
第165条
第166条
第167条
第168条
第169条
第170条
第171条
第172条
第173条
第174条
第175条
第176条
第177条
第178条
第179条
第180条
第181

OperationalError: table movie250 has no column named oname

In [3]:
import urllib
from bs4 import BeautifulSoup
url = "https://www.doubai.com" 
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75"
}
req = urllib.request.Request(url=url,headers=headers)
html = urllib.request.urlopen(req)
#测试bs4包的使用
#beautiful将复杂的html文档转换成一个复杂的树形结构，每个节点分为四种
#1.Tag 标签及其内容：拿到它找到的第一个内容
#2.NavigableString 标签里的内容
#3.attrs 标签中的属性
#BeautifulSoup 表示整个文档
#print(bs.name)
#4.获得整个文档的内容
#Comment 指的是文档中不含注释的内容
#文档的遍历和搜索
bs = BeautifulSoup(html,"html.parser")

In [14]:
import re
#更多内容搜索文档
# print(bs.head.contents[1])
#基于文档的搜索，查询操作
#(1) find_all() 字符串过滤
t_list = bs.find_all("src")
# print(t_list)
#正则表达式搜索使用search()方法来匹配
t_list = bs.find_all(re.compile("a"))
print(t_list)

[<head>
<meta charset="utf-8"/>
<title></title>
<link href="//dragonstatic.com/parking/css/native.css" media="screen" rel="stylesheet" type="text/css"/>
<script src="//dragonstatic.com/parking/js/track.js?v=190929" type="3a403e5efd17e3e78e5868b9-text/javascript"></script>
<script type="3a403e5efd17e3e78e5868b9-text/javascript">var pageinfo = {ad:1};</script>
<script src="//dragonstatic.com/parking/js/native.js?v=201206" type="3a403e5efd17e3e78e5868b9-text/javascript"></script>
</head>, <meta charset="utf-8"/>]


In [1]:
import sqlite3
conn = sqlite3.connect("test.db")#打开或创建数据库文件
print("Open database successfully!")
c = conn.cursor() #获取游标
sql = '''
    create table company
        (id int primary key not null,
        name text not null,
        age int not null,
        address char(50),
        salary real);
''' #执行sql语句
c.execute(sql)
conn.commit()#提交数据库操作
conn.close()#关闭数据库操作

Open database successfully!


In [2]:
#插入数据
conn = sqlite3.connect("test.db")
print("成功打开数据库")
c = conn.cursor()
sql1 = '''
    insert into company (id,name,age,address,salary)
    values(1,'张三',32,"成都",8000);   
''' #执行sql1语句
sql2 = '''
    insert into company (id,name,age,address,salary)
    values(2,'李四',30,"重庆",15000);   
''' #执行sql2语句
c.execute(sql1)
c.execute(sql2)
conn.commit()#提交数据库操作
conn.close()#关闭数据库操作



成功打开数据库


In [3]:
#查询数据
import sqlite3
conn = sqlite3.connect("test.db")
print("成功打开数据库")
c = conn.cursor()
sql = "select id,name,address,salary from company" #执行sql语句
cursor=c.execute(sql)
for row in cursor:
    print("id=",row[0])
    print("name=",row[1])
    print("address=",row[2])
    print("salary=",row[3],"\n")
conn.commit()#提交数据库操作
conn.close()#关闭数据库操作
print("查询完毕")

成功打开数据库
id= 1
name= 张三
address= 成都
salary= 8000.0 

id= 2
name= 李四
address= 重庆
salary= 15000.0 

查询完毕


In [6]:
import sqlite3
conn = sqlite3.connect("moive.db")
cur = conn.cursor()
sql = '''insert into movie250(info_link,pic_link,cname,oname,score,rated,introduction,info)
                values("https://movie.douban.com/subject/1292052/","https://img2.doubanio.com/view/photo/s_ratio_poster/public/p480747492.jpg","肖申克的救赎"," ",9.7,123,"希望让人自由","导演: 弗兰克·德拉邦特 Frank Darabont   主演: 蒂姆·罗宾斯 Tim Robbins  ... 1994   美国   犯罪 剧情")'''
cur.execute(sql)
cur.commit()
cur.close()

OperationalError: no such table: movie250